In [1]:
import graphlab
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
sales = graphlab.SFrame('kc_house_data_small.gl/')

This non-commercial license of GraphLab Create for academic use is assigned to sripriya.kuram@gmail.com and will expire on September 13, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\sripr\AppData\Local\Temp\graphlab_server_1479532753.log.0


In [3]:
(train_and_validation, test) = sales.random_split(.8, seed=1)
(train, validation) = train_and_validation.random_split(.8, seed=1)

In [4]:
def get_numpy_data(data_sframe, features, output):
    data_sframe['constant'] = 1
    features_list = ['constant'] + features
    feature_frame = data_sframe[features_list]
    feature_matrix = feature_frame.to_numpy()
    output_array = data_sframe[output].to_numpy()
    return (feature_matrix, output_array)

In [5]:
def normalize_features(features):
    norms = np.linalg.norm(features , axis = 0)
    normalized_features = features / norms
    return (normalized_features, norms)

In [8]:
feature_list = ['bedrooms',  
                'bathrooms',  
                'sqft_living',  
                'sqft_lot',  
                'floors',
                'waterfront',  
                'view',  
                'condition',  
                'grade',  
                'sqft_above',  
                'sqft_basement',
                'yr_built',  
                'yr_renovated',  
                'lat',  
                'long',  
                'sqft_living15',  
                'sqft_lot15']
features_train, output_train = get_numpy_data(train, feature_list, 'price')
features_test, output_test = get_numpy_data(test, feature_list, 'price')
features_valid, output_valid = get_numpy_data(validation, feature_list, 'price')

In [9]:
features_train, norms = normalize_features(features_train) 
features_test = features_test / norms 
features_valid = features_valid / norms

In [10]:
query_house = features_test[0]

In [11]:
query_house

array([ 0.01345102,  0.01551285,  0.01807473,  0.01759212,  0.00160518,
        0.017059  ,  0.        ,  0.05102365,  0.0116321 ,  0.01564352,
        0.01362084,  0.02481682,  0.01350306,  0.        ,  0.01345386,
       -0.01346927,  0.01375926,  0.0016225 ])

In [23]:
np.sqrt(np.sum((query_house - features_train[9])**2))

0.059723593716661257

In [22]:
#eatures_train[0:9]
distances1 = []
for i in range(10):
    distances1.append(np.sqrt(np.sum((query_house - features_train[i])**2)))
min(distances1)
distances1
    

[0.0602747091729555,
 0.085468811488270832,
 0.061499464371202843,
 0.053402739788200579,
 0.058444840639381393,
 0.059879215101840008,
 0.054631404972615261,
 0.055431083241597921,
 0.052383627840972731,
 0.059723593716661257]

In [26]:
diff = features_train[0:len(features_train)] - query_house

In [31]:
distances = np.sqrt(np.sum((diff**2),axis=1))
len(distances)

5527

In [32]:
print distances[100]

0.0237082324496


In [43]:
def compute_distances(features_instances, features_query):
    diff = features_instances[0:len(features_instances)] - features_query
    distances = np.sqrt(np.sum((diff**2),axis=1))
    return distances

In [44]:
x = compute_distances(features_train, features_test[2])

In [45]:
np.where(x == x.min())

(array([382], dtype=int64),)

In [46]:
train[382]['price']

249000L

In [47]:
def k_nearest_neighbors(k, feature_train, features_query):
    diff = feature_train[0:len(feature_train)] - features_query
    distances = np.sqrt(np.sum((diff**2),axis=1))
    distances = np.argsort(distances)
    neighbors = distances[0:k]    
    return neighbors

In [63]:
y = k_nearest_neighbors(4, features_train, features_test[2])
y

array([ 382, 1149, 4087, 3142], dtype=int64)

In [166]:
def predict_output_of_query(k, feature_train, output_train, features_query):
    avg_price= 0
    diff = feature_train[0:len(feature_train)] - features_query
    distances = np.sqrt(np.sum((diff**2),axis=1))
    distances = np.argsort(distances)
    neighbors = distances[0:k]
    print neighbors
    for i in neighbors:
        avg_price += output_train[i]
    prediction = avg_price/len(neighbors)
        
    return prediction

In [167]:
z = predict_output_of_query(4, features_train, output_train, features_test[2])
z

[ 382 1149 4087 3142]


413987

In [178]:
def predict_output(k, feature_train, output_train, features_query):
    avg_price= 0
    final_prediction =[]
    for j in features_query:
        diff = feature_train[0:] - j
        distances = np.sqrt(np.sum((diff**2),axis=1))
        distances = np.argsort(distances)
        neighbors = distances[0:k]
        #print neighbors
        avg_price = 0
        for i in neighbors:
            avg_price += output_train[i]
        prediction = avg_price/len(neighbors)
        #print prediction
        final_prediction.append(prediction)
    return final_prediction

In [192]:
rss_all = []
for k in range(1,16):
    fpp = predict_output(k, features_train, output_train, features_valid[0:len(features_valid)])
    rss = (output_valid - fpp)**2
    rss_all.append(np.sum(rss,axis=0))
print rss_all.index(min(rss_all))
    
print rss_all

7
[105453830251561, 83445073036317, 72692113667429, 71946729679356, 69846516027353, 68899557352960, 68341986155264, 67361700020962, 68372752298062, 69335060576786, 69523884081979, 69049991793212, 70011272351177, 70908725305267, 71106956486642]


In [197]:
fp2 = predict_output(8, features_train, output_train, features_test)
rss = (output_test - fp2)**2
rss_all.append(np.sum(rss,axis=0))
rss_all

[105453830251561,
 83445073036317,
 72692113667429,
 71946729679356,
 69846516027353,
 68899557352960,
 68341986155264,
 67361700020962,
 68372752298062,
 69335060576786,
 69523884081979,
 69049991793212,
 70011272351177,
 70908725305267,
 71106956486642,
 133118842702196,
 133118842702196]

In [175]:
fp = predict_output(10, features_train, output_train, features_test[0:10])
print min(fp)
fp.index(min(fp))

[3758 3144 4208 5038 1495 4614 1389  523 4893 1659]
881300
[4670 1385 4834 2226  887   57  276  635 2719  736]
431860
[ 382 1149 4087 3142 2751 4556 3372 4584 1666 2283]
460595
[1152 4263 2740 4720 2103 1636 2723 4864 4141 1044]
430200
[ 343 1521 2749 4721 4461 2894 3238 2782 3043  106]
766750
[3538 2682 2179 3165 2543 4403 3585  356 3174 5231]
667420
[3290 3483 3374 4470 1729 3229 5110 5486  856 1685]
350032
[2642 2813 3803   18 1728 1822 2891 4417 4056 2865]
512800
[ 159 4078 3312  261 4863 1466 1866  978  560 3191]
484000
[1291 4749 1441 4571 2402 2639 5157 5514 4737 3361]
457235
350032


6

In [173]:

for j in features_test[0:1]:
    print j
print features_test[0]
    

[401]
215000
[901]
252000
[4979]


IndexError: index 4979 is out of bounds for axis 0 with size 1435

In [131]:
q= [1,2,3,4,5]
w =[2,3,4,5,6]
for i in q:
    print i
    print w[q.index(i)]
    print "printing rss"
    rss += ((i - w[q.index(i)])**2)
    print (rss)

1
2
printing rss
2
2
3
printing rss
3
3
4
printing rss
4
4
5
printing rss
5
5
6
printing rss
6
